# **Data Cleaning Notebook**

## Objectives

* Clean data
* Split cleaned dataset into Train and Test sets

## Inputs

* outputs/datasets/collection/LoanDefaultData.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned


---

# Imports

In [ ]:
import os
import pandas as pd
# for vs code
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from feature_engine.outliers import Winsorizer
from feature_engine.imputation import MeanMedianImputer

---

# Change working directory

We need to change the working directory from its current folder, where the notebook is stored, to its parent folder
* First we access the current directory with os.getcwd()

In [ ]:
current_dir = os.getcwd()
current_dir

* Then we want to make the parent of the current directory the new current directory
    * os.path.dirname() gets the parent directory
    * os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
print(f"You set a new current directory: {current_dir}")

---

# Load Data

In [ ]:
df = pd.read_csv("outputs/datasets/collection/LoanDefaultData.csv")
df.head(3)

# Data Cleaning

Set the target variable

In [ ]:
target_var = "loan_status"

## Drop duplicate entries

In the data collection step we already established that there are some duplicated entries in the dataset. As they account to less than 1 % of data we will drop them from the dataset.

Show duplicated entries:

In [ ]:
duplicates = df.duplicated()
df[df.duplicated(keep=False)].sort_values(by=['person_age','person_income'])

Duplicates should be dropped before splitting into training and test sets to prevent data leakage, which could artificially inflate model performance. Removing duplicates beforehand also ensures that both sets reflect the true data distribution and that evaluation metrics remain reliable.

No duplicated entries remain after dropping them:

In [ ]:
df = df.drop_duplicates()
df[df.duplicated()]

## Split Train and Test Set

Missing data imputation and outlier treatment should be done after splitting into training and test sets to avoid data leakage. Therefore we now split the data into train and test set.

In [ ]:
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df[target_var],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

## Missing Data

We first check for missing data in the train set and can confirm that there is some missing data present in the dataset.

In [ ]:
def show_missing_values(df):
    print("Number of missing values in each column:")

    missing_count = df.isna().sum()
    missing_percent = (df.isna().sum() / len(df)) * 100

    missing_data = pd.DataFrame({
        'Missing Values': missing_count,
        'Percentage': missing_percent.round(2)
    })

    print(missing_data)

    print("\nTotal number of missing values in the dataframe:", 
          df.isna().sum().sum())
    
show_missing_values(TrainSet)

* The Train Set contains missing values in the variables `person_emp_length` and `loan_int_rate`, both with less than 10% of observations missing

Given the relatively small proportion of missing data, these values will be handled using **median imputation**. This approach is appropriate since the numerical features are skewed and contain outliers, making the median a more robust measure than the mean.

In [ ]:
imputer = MeanMedianImputer(imputation_method='median', 
                            variables=['person_emp_length', 'loan_int_rate'])
df_cleaned = imputer.fit_transform(TrainSet)

show_missing_values(df_cleaned)

Compare distributions before and after imputing the missing values:

In [ ]:
print("Before imputing missing values:")
missing_cols = ['person_emp_length', 'loan_int_rate']
TrainSet[missing_cols].describe().T

In [ ]:
print("After imputing missing values:")
df_cleaned[missing_cols].describe().T

The summary statistics between the original training dataset and the cleaned dataset are very similar, indicating that the distributions have not been significantly affected by the median imputation. This confirms that the imputation preserved the overall data characteristics, so the same median imputer will now be applied to both the training and test sets.

In [ ]:
imputer = MeanMedianImputer(imputation_method='median', 
                            variables=['person_emp_length', 'loan_int_rate'])
TrainSet = imputer.fit_transform(TrainSet)
TestSet = imputer.transform(TestSet)

# Outliers

In the previous exploratory analysis, we observed that several numerical variables contained pronounced outliers, mostly in the upper range of their distributions.
To mitigate their influence on model performance while preserving the overall data structure, we apply Winsorization on the right tail.

This approach caps extreme values at defined thresholds (based on the interquartile range), reducing the impact of outliers without removing observations from the dataset.

For `person_income`, `person_age`, and `person_emp_length`, we observed some values that are clearly not realistic. Other numerical variables also show outliers, but they lie within a credible range, so trimming them is not strictly necessary as it would remove potentially useful information. Depending on the model choice, this decision can be revisited; for example, linear models might benefit from a tighter fold to reduce the influence of extreme values.

We use a fold of 5 for Winsorization to cap only the most extreme outliers. This way we only target outliers in the variables `person_income`, `person_age`, and `person_emp_length`.

Apply Winsorization:

In [ ]:
numeric_cols = (TrainSet
                .select_dtypes(include=['float64', 'int64'])
                .columns
                .drop("loan_status").tolist())

winsorizer = Winsorizer(capping_method='iqr', fold=5, 
                        tail='right', variables=numeric_cols)
df_winsorized = winsorizer.fit_transform(TrainSet)

Compare differences:

In [ ]:
def count_outliers(series):
    """Return the number of outliers in a pandas Series using the IQR method."""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 5 * IQR
    upper_bound = Q3 + 5 * IQR
    return ((series < lower_bound) | (series > upper_bound)).sum()

# Compute outlier counts for each feature before and after cleaning
outliers_before = {col: count_outliers(TrainSet[col]) for col in numeric_cols}
outliers_after = {
    col: count_outliers(df_winsorized[col]) 
    for col in numeric_cols
    }

# Combine results into a single DataFrame (one column per feature)
outlier_comparison = pd.DataFrame(
    [outliers_before, outliers_after], 
    index=['Outliers Before Cleaning', 'Outliers After Cleaning']).T

outlier_comparison

In [ ]:
n_cols = 2  
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols 
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*5, n_rows*2))
axes = axes.flatten()  

for i, col in enumerate(numeric_cols):
    sns.boxplot(x=df_winsorized[col], ax=axes[i], 
                color=sns.color_palette("Set2")[0])
    axes[i].set_title(f"{col}")

# Remove any unused subplots
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

The outlier comparison and boxplots show that all extreme values in the numerical features were effectively removed after applying Winsorization on the right tail. This confirms that the Winsorizer successfully capped high-end outliers without altering the overall data structure. Based on these results, we will apply the same Winsorization procedure to both the training and test sets to ensure consistent preprocessing across the entire modeling pipeline.

In [ ]:
winsorizer = Winsorizer(capping_method='iqr', fold=5, 
                        tail='right', variables=numeric_cols)
TrainSet = winsorizer.fit_transform(TrainSet)
TestSet = winsorizer.transform(TestSet)

---

# Push files to Repo

In [ ]:
file_path = 'outputs/datasets/cleaned'

try:
    os.makedirs(name=file_path)
except Exception as e:
    print(e)

# Save the Train and Test sets as csv files for further use
filename = "TrainSet.csv"
TrainSet.to_csv(f"{file_path}/{filename}", index=False)

filename = "TestSet.csv"
TestSet.to_csv(f"{file_path}/{filename}", index=False)

---

# Conclusions and Next Steps

We performed key data cleaning steps including:
* Removal of duplicate rows
* Median imputation for missing values
* Winsorization of extreme outliers 

Further data cleaning actions are not required, as all columns could potentially have predictive power, no columns need removal, and categorical values were consistent in the exploratory analysis.

Next Steps:
* Prepare data for feature engineering and modeling